# Clustering
## Author: Zhenggang Tan
- The goal of this notebook is to try out different clustering methods on the cleaned data.
- Below is an example of:
1. Vectorization of each article(abstract) to Word embedding vectors using Term Frequency and Inverse Document Frequency(TF-IDF).
2. Clustering using K-means.
3. Dimension reduction using t-distributed Stochastic Neighbor Embedding(t-SNE).
- And finally make a plot combining cluster labels and dimension-reduced document.

### Import Packages

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
import warnings
import faiss
from scipy.spatial.distance import cdist
from sklearn.manifold import TSNE
import seaborn as sns
from sklearn.decomposition import TruncatedSVD

%matplotlib inline
warnings.filterwarnings('ignore')
plt.style.use('ggplot')

### Read in Data
- The data will be too large to upload to GitHub. Please make sure you have the right directory set for the `read_csv` function.

In [ ]:
df_en_abstract_only = pd.read_csv('../Data/article_proc.csv')

In [ ]:
df_en_abstract_only.head()

### Define vectorization technique
- Here we used TF-IDF as Vectorization technique.

TODO: TF-IDF short explanation

Tf-idf stands for term frequency-inverse document frequency, and the tf-idf weight is a weight often used in information retrieval and text mining. This weight is a statistical measure used to evaluate how important a word is to a document in a collection or corpus. The importance increases proportionally to the number of times a word appears in the document but is offset by the frequency of the word in the corpus. 

Term Frequency, which measures how frequently a term occurs in a document.

TF(t) = (Number of times term t appears in a document) / (Total number of terms in the document)

Inverse Document Frequency, which measures how important a term is. While computing TF, all terms are considered equally important. However it is known that certain terms, such as "is", "of", and "that", may appear a lot of times but have little importance. Thus we need to weigh down the frequent terms while scale up the rare ones.

IDF(t) = log_e(Total number of documents / Number of documents with term t in it)




- We hope eventually we could try out different vectorization techniques and find which one is most suitable for this task.
TODO: implement `Word2Vec`, `FastText`, `BERT` version of word embedding technique.

In [ ]:
def vectorize(text, maxx_features):
    vectorizer = TfidfVectorizer(max_features=maxx_features).fit(text)
    X = vectorizer.transform(text)
    return X

In [ ]:
# Use abstract for embedding otherwise will take too long
abstract = df_en_abstract_only['abstract'].values
# Set maximum for number of features we want.
# Although the more feature the richer the content
# You don't want it to be too large because our
# Dataset has around 240000 samples, and later when
# We want to fit clustering algorithms we have to
# Turn the word embeddings to numpy array(either np.float64
# or np.float32) which requires 240000*max_features*4 bytes in
# Memory. 4096 feature requires about 3.66 GB on RAM! Consider
# Changing this value to a smaller one if you run into memory error.
max_features = 2**12

X = vectorize(abstract, max_features)

### Clustering
- Below is our demo clustering using K-means.
Selected `K=40` according to distortions of `K` from 10 to 50(Could try more).
- TODO: Implement with more clustering techniques, possible ones are:
1. Spectral clustering
2. DBSCAN(might not work well since it's heavily used for outlier detection).
3. Gaussian Mixture Model
4. BIRCH
5. Affinity Propagation clustering
6. Mean-Shift clustering
7. OPTICS algorithm
8. Agglomerative Hierarchy clustering
See this link for explanations and code: [Clustering](https://www.freecodecamp.org/news/8-clustering-algorithms-in-machine-learning-that-all-data-scientists-should-know/)
Ideally, we should select the same number of clusters for each algorithm, so we could compare them on the same scale.
<strong>Caution!</strong>: There are many packages out there that might out-perform `scikit-learn` on a specific task(i.e. Facebook's `Faiss` for K-means and PCA). Considering the scope of our dataset, try what's the best to save some time!

In [ ]:
X_32 = X.todense().astype('float32').toarray()
n_init = 5
max_iter = 100
distortions = []

# run kmeans with many different k
K = range(10, 50)
for k in K:
    print('fitting clusters with {} clusters'.format(k))
    k_means = faiss.Kmeans(d=X_32.shape[1], k=k, niter=max_iter, nredo=n_init,gpu=True,verbose=True,seed=553602)
    k_means.train(X_32)
    distortions.append(sum(np.min(cdist(X_32, k_means.centroids, 'euclidean'), axis=1)) / X.shape[0])
    print('Found distortion for {} clusters'.format(k))

In [ ]:
# Plot to choose a best K
X_line = [K[0], K[-1]]
Y_line = [distortions[0], distortions[-1]]

# Plot the elbow
plt.plot(K, distortions, 'b-')
plt.plot(X_line, Y_line, 'r')
plt.xlabel('k')
plt.ylabel('Distortion')
plt.title('The Elbow Method showing the optimal k')
plt.show()

In [ ]:
# Chose 40 as best K, rerun with K=40 and predict label for each abstract(on the word embeddings).
k = 40
k_means = faiss.Kmeans(d=X_32.shape[1], k=k, niter=max_iter, nredo=n_init,gpu=True,verbose=True,seed=553602)
k_means.train(X_32)
label = k_means.assign(X_32)

In [ ]:
# Assign clusters to dataset.
# CHANGE THE VARIABLE NAME TO SOMETHING ELSE! DO NOT OVERWRITE!
df_en_abstract_only['cluster'] = label[1]

### Dimension Reduction for plotting
- Our goal is to have some nice visualization of articles that are similar to be plotted together somehow.
- How? Dimension reduction! Specifically, we reduce the dataset's feature number to 2(2D) or 3(3D), so we can construct visualizations.
- TODO: t-SNE explanation
See this link for paper link and implementation: [t-SNE](https://lvdmaaten.github.io/tsne/)
- There could be other techniques that do this but this is the best algorithm so far.

In [ ]:
tsne = TSNE(verbose=1, perplexity=50)  # Changed perplexity from 100 to 50 per FAQ
X_embedded = tsne.fit_transform(X_32)

In [ ]:
# Visualization without coloring using labels
# sns settings
sns.set(rc={'figure.figsize':(15,15)})

# colors
palette = sns.color_palette("bright", 1)

# plot
sns.scatterplot(X_embedded[:,0], X_embedded[:,1], palette=palette)
plt.title('t-SNE with no Labels')
plt.savefig("t-sne_covid19.png")
plt.show()

In [ ]:
# Visualization using labels
# sns settings
sns.set(rc={'figure.figsize':(20,20)})

# colors
palette = sns.hls_palette(40, l=.4, s=.9)

# plot
sns.scatterplot(X_embedded[:,0], X_embedded[:,1], hue=label[1], legend='full', palette=palette)
plt.title('t-SNE with Kmeans Labels')
plt.savefig("improved_cluster_tsne.png")
plt.show()

### <strong>OPTIONAL BUT HIGHLY RECOMMENDED</strong>
- You have seen our visualization and probably find it to be an awful clustering.
- Why? We used all of our features to cluster, but not all of them are useful. In fact, most of them would be useless if we think in a way of Principal Component Analysis. However, our word embedding matrix is a sparse matrix which does not work well with PCA, so we have to use `Truncated SVD` to truncate our features to `n` dimensions that explains the most variances. From there, we fit our clustering algorithms.

In [ ]:
X_reduced = TruncatedSVD(n_components=50, random_state=553602).fit_transform(X_32)

In [ ]:
k = 40
k_means = faiss.Kmeans(d=X_reduced.shape[1], k=k, niter=max_iter, nredo=n_init,gpu=True,verbose=True,seed=553602)
k_means.train(X_reduced)
label_reduced = k_means.assign(X_reduced)

In [ ]:
tsne_reduced = TSNE(verbose=1, perplexity=50)  # Changed perplexity from 100 to 50 per FAQ
X_embedded_reduced = tsne_reduced.fit_transform(X_reduced)

In [ ]:
# sns settings
sns.set(rc={'figure.figsize':(20,20)})

# colors
palette = sns.hls_palette(40, l=.4, s=.9)

# plot
sns.scatterplot(X_embedded_reduced[:,0], X_embedded_reduced[:,1], hue=label_reduced[1], legend='full', palette=palette)
plt.title('t-SNE with Kmeans Labels')
plt.savefig("improved_cluster_tsne.png")
plt.show()

This plot now makes much more sense. Later we will perform topic modeling on each of the clusters.